# Configuration of the GeoipEnricher

This presentations goal is to introduce the configuration of the output subfields of the `GeoipEnricher`.

Prerequisites: a local geo ip database is available

### The challenge

The given document

In [92]:
document = {"client": {"ip": "8.8.8.8"}}


will result in the default output

In [93]:
default_output = {
    "client": {"ip: 8.8.8.8"},
    "geoip": {
        "geometry": {"coordinates": [-97.822, 37.751], "type": "Point"},
        "properties": {
            "accuracy_radius": 1000,
            "continent": "North America",
            "continent_code": "NA",
            "country": "United States",
            "country_iso_code": "US",
            "time_zone": "America/Chicago",
        },
        "type": "Feature",
    },
}


which instead should be configured to look like

In [94]:
expected_output = {
    "client": {
        "geo": {
            "accuracy": 1000,
            "continent_code": "NA",
            "continent_name": "North America",
            "country_iso_code": "US",
            "country_name": "United States",
            "geometry_type": "Point",
            "location": [-97.822, 37.751],
            "timezone": "America/Chicago",
            "type": "Feature",
        },
        "ip": "8.8.8.8",
    }
}


### Create rule and processor
create the rule:

In [95]:
from pathlib import Path
import tempfile


rule_yaml = """---
filter: "client.ip"
geoip_enricher:
  source_fields: ["client.ip"]
  customize_target_subfields: 
    type: client.geo.type
    geometry.type: client.geo.geometry_type
    geometry.coordinates: client.geo.location
    properties.accuracy_radius: client.geo.accuracy
    properties.continent: client.geo.continent_name
    properties.continent_code: client.geo.continent_code
    properties.country: client.geo.country_name
    properties.city: client.geo.city_name
    properties.postal_code: client.geo.postal_code
    properties.subdivision: client.geo.subdivision
    properties.time_zone: client.geo.timezone
    properties.country_iso_code: client.geo.country_iso_code
"""

rule_path = Path(tempfile.gettempdir()) / "geoip"
rule_path.mkdir(exist_ok=True)
rule_file = rule_path / "data-stream.yml"
rule_file.write_text(rule_yaml)


678

create the processor config and replace the `db_path` with your local geo ip database:

In [96]:
processor_config = {
    "geoip_enricher": {
        "type": "geoip_enricher",
        "specific_rules": [str(rule_path)],
        "generic_rules": ["/dev"],
        "db_path": "<INSERT_PATH_TO_GEOIP_DATABASE>"
    }
}


create the processor with the factory:

In [97]:
from unittest import mock
import sys
sys.path.append("..")
from logprep.factory import Factory

mock_logger = mock.MagicMock()
geoip_enricher = Factory.create(processor_config, mock_logger)
geoip_enricher


geoip_enricher

### Process event

In [98]:
from copy import deepcopy

mydocument = deepcopy(document)
geoip_enricher.process(mydocument)
assert mydocument == expected_output
f"The output has the expected form: {mydocument == expected_output}"

'The output has the expected form: True'